
# Closed-Loop Flow-Control Strategies in **TRUST**

This Jupyter notebook demonstrates how to model a **closed hydraulic circuit** in TRUST code while *actively imposing a target volumetric flow-rate*. Three alternative source-term strategies are benchmarked side-by-side.

---
## Control Strategies Under Test
| Label | Source-term combination | Regulated parameter | Control objective |
|-------|------------------------|---------------------|-------------------|
| **`dp_k_regul`** | ➤ `DP_Impose` (fixed ΔP)<br>➤ `Perte_Charge_Singuliere` | Loss-coefficient **K** | Tune **K** so that *Q → Q<sub>target</sub>* |
| **`dp_regul`** | ➤ `DP_Impose` with built-in regulator | Imposed **ΔP** | Tune **ΔP** so that *Q → Q<sub>target</sub>* |
| **`pcr_regul`** | ➤ `DP_Impose` (fixed ΔP)<br>➤ `Perte_charge_isotrope` (regular head loss) | Correction factor **λ_corr** | Tune **λ_corr** so that *Q → Q<sub>target</sub>* |

> **Note** All three methods use the same target (\(Q_{\text{target}} = 5\;\text{m}^3\,\text{h}^{-1}\) here) and share identical “open-loop” conditions so that differences come solely from the control law.

---
## Numerical Test Matrix
```python
cases = {
    "ns"   : ["PolyMAC_P0P1NC", "PolyMAC_P0", "VEFPreP1b"],
    "multi": ["PolyMAC_P0P1NC", "PolyMAC_P0"]
}
```
For each **physics family** (`ns` = single-phase Navier–Stokes, `multi` = multiphase model) we loop over the selected discretisations.

A last test is added for the time dependant imposed $\Delta P$, where the term vanishes to 0 between 20 and 21s (like for a pump trip).
## Domain

![](src/domaine.png)

In [ ]:
from trustutils import run
run.introduction("A. Gerschenfeld, Y. Gorsse")
run.useMEDCoupling()
from src.bmesh import build_mesh

run.reset()
cases = {"ns" : ["PolyMAC_P0P1NC", "PolyMAC_P0", "VEFPreP1b"], "multi" : ["PolyMAC_P0P1NC", "PolyMAC_P0"]}
sources = {"dp_k_regul": "DP_Impose { dp Champ_uniforme 3 100 0 0 surface { face_group surf2 orientation Champ_Uniforme 3 0 0 -1 } } , perte_Charge_Singuliere { dir K regul { K0 7.0 deb -5 eps 10*(t>1) } surface { face_group surf1 orientation Champ_Uniforme 3 0 0 -1 } }",
           "dp_regul": "DP_Impose { dp_regul { DP0 100 deb 5 eps 10*(t>1) } surface { face_group surf2 orientation Champ_Uniforme 3 0 0 -1 } }"}

run.initBuildDirectory()
for nom_s, s in sources.items():
    for pb, dis in cases.items():
        for d in dis:
            run.addCaseFromTemplate(f"jdd_{pb}.data", f"{pb}/{d}/{nom_s}",
                                    {"dis" : d,
                                     "sources" : s,
                                     "poly" : "" if d == "VEFPreP1b" else "convertalltopoly"
                                     })
            build_mesh(d == "VEFPreP1b", f"{run.BUILD_DIRECTORY}/mesh_hexa.med", f"{run.BUILD_DIRECTORY}/{pb}/{d}/{nom_s}/mesh.med")

# add case with regulated regular pressure drop (not in the main loop to keep the previous cases order for the NR tests)
nom_s, s = "pcr_regul", "DP_Impose { dp Champ_uniforme 3 100 0 0 surface { face_group surf2 orientation Champ_Uniforme 3 0 0 -1 } } , Perte_charge_isotrope { lambda 7 diam_hydr Champ_uniforme 1 0.5 regul { K0 1.0 deb -5 eps t>1 } surface { face_group surf1 orientation Champ_Uniforme 3 0 0 -1 } }"
for pb, dis in cases.items():
        for d in dis:
            if d == "VEFPreP1b":
                continue
            run.addCaseFromTemplate(f"jdd_{pb}.data", f"{pb}/{d}/{nom_s}",
                                    {"dis" : d,
                                     "sources" : s,
                                     "poly" : "" if d == "VEFPreP1b" else "convertalltopoly"
                                     })
            build_mesh(d == "VEFPreP1b", f"{run.BUILD_DIRECTORY}/mesh_hexa.med", f"{run.BUILD_DIRECTORY}/{pb}/{d}/{nom_s}/mesh.med")

sources[nom_s] = s

# add a last case to test the DP(t) feature
nom_s, s = "dp_t", "DP_Impose { dp_regul { DP0 50*(1_min_(0_max_(20-t))) deb 5 eps (t>1)*(t<20) } surface { face_group surf2 orientation Champ_Uniforme 3 0 0 -1 } }"
for pb, dis in cases.items():
        for d in dis:
            # if d == "VEFPreP1b":
            #     continue
            run.addCaseFromTemplate(f"jdd_{pb}.data", f"{pb}/{d}/{nom_s}",
                                    {"dis" : d,
                                     "sources" : s,
                                     "poly" : "" if d == "VEFPreP1b" else "convertalltopoly"
                                     })
            build_mesh(d == "VEFPreP1b", f"{run.BUILD_DIRECTORY}/mesh_hexa.med", f"{run.BUILD_DIRECTORY}/{pb}/{d}/{nom_s}/mesh.med")

sources[nom_s] = s
run.printCases()
run.runCases()

In [ ]:
from matplotlib import markers
from trustutils import plot
discretization_colors = {
    "PolyMAC_P0P1NC": "blue",
    "PolyMAC_P0": "green",
    "VEFPreP1b": "red"
}
pb_markers = {
    "ns": "-",
    "multi": "--"
}

a = plot.Graph("DP impose, Ksing regul", nY=2)
nom_s = "dp_k_regul"
s = sources[nom_s]

for pb, dis in cases.items():
    for i, d in enumerate(dis):
        a.addPlot(0)
        data = plot.loadText(f"{pb}/{d}/{nom_s}/jdd_{pb}_pb_K_surf1.out", skiprows=3)
        a.add(data[0], data[1], label=f"{pb} - {d}", marker=pb_markers[pb], color=discretization_colors[d])
        a.label("Time [s]", "Coeff de perte de charge")
        a.addPlot(1)
        a.add(data[0], data[2], label=f"{pb} - {d}", marker=pb_markers[pb], color=discretization_colors[d])
        if i == 0 and pb == "ns": a.add(data[0], data[3], label="debit cible", color="black")
        a.label("Time [s]", "Flow rate [kg/s]")

a = plot.Graph("DP regul", nY=2)
nom_s = "dp_regul"
s = sources[nom_s]

for pb, dis in cases.items():
    for i, d in enumerate(dis):
        data = plot.loadText(f"{pb}/{d}/{nom_s}/jdd_{pb}_pb_DP_surf2.out", skiprows=3)
        a.addPlot(0)
        a.add(data[0], data[1], label=f"{pb} - {d}", marker=pb_markers[pb], color=discretization_colors[d])
        a.label("Time [s]", "Imposed pressure difference [Pa]")
        a.addPlot(1)
        a.add(data[0], data[2], label=f"{pb} - {d}", marker=pb_markers[pb], color=discretization_colors[d])
        if i == 0 and pb == "ns": a.add(data[0], data[3], label="debit cible", color="black")
        a.label("Time [s]", "Flow rate [kg/s]")

a = plot.Graph("DP impose, PCR regul", nY=2)
nom_s = "pcr_regul"
s = sources[nom_s]

for pb, dis in cases.items():
    for i, d in enumerate(dis):
        if d == "VEFPreP1b":
                continue
        data = plot.loadText(f"{pb}/{d}/{nom_s}/jdd_{pb}_pb_DP_dom.out", skiprows=3)
        a.addPlot(0)
        a.add(data[0], data[1], label=f"{pb} - {d}", marker=pb_markers[pb], color=discretization_colors[d])
        a.label("Time [s]", "facteur de regulation")
        a.addPlot(1)
        a.add(data[0], data[2], label=f"{pb} - {d}", marker=pb_markers[pb], color=discretization_colors[d])
        if i == 0 and pb == "ns": a.add(data[0], data[3], label="debit cible", color="black")
        a.label("Time [s]", "Flow rate [kg/s]")

a = plot.Graph("DP(t)", nY=2)
nom_s = "dp_t"
s = sources[nom_s]

for pb, dis in cases.items():
    for i, d in enumerate(dis):
        data = plot.loadText(f"{pb}/{d}/{nom_s}/jdd_{pb}_pb_DP_surf2.out", skiprows=3)
        a.addPlot(0)
        a.add(data[0], data[1], label=f"{pb} - {d}", marker=pb_markers[pb], color=discretization_colors[d])
        a.label("Time [s]", "Imposed pressure difference [Pa]")
        a.addPlot(1)
        a.add(data[0], data[2], label=f"{pb} - {d}", marker=pb_markers[pb], color=discretization_colors[d])
        if i == 0 and pb == "ns": a.add(data[0], data[3], label="debit cible", color="black")
        a.label("Time [s]", "Flow rate [kg/s]")